In [1]:
import datetime
import ujson
import numpy 
data = ujson.load(open('user_project_times.json'))

In [2]:
summaries = {}
ctr = 0
for line in open('user_project_summaries.json'): # lazy iteration because the file is large
    print ctr,
    ctr+=1
    summaries.update(ujson.loads(line))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46


In [88]:
TASK = "RETENTION"
if TASK == 'RETENTION': 
    data = old_data
if TASK == 'LANGUAGE':
    data = lang_data

Create Language Data Set and Initialize Language Codes

In [4]:
'''We are using all the data from the user from whom we we have previously 
classfied what their primary langauge is '''
user_langs = ujson.load(open('user_inferredlangs.json')) 
lang_data = {}
keys = user_langs.keys()

print 'done loading keys'
for key in keys: 
    lang_data[key] = data[key[0:2] + '/' + key]
print 'lang data created'

lang_dict = {} #key is langauge isocode, value is number 
isocodes = ujson.load(open('isocodes.json'))
counter = 0 
for lang in isocodes:
    lang_dict[lang] = counter
    counter+=1
    

done loading keys
lang data created


In [5]:
def convert_time(el):
    el = int(str(el)[:10])
    return datetime.date.fromtimestamp(el)

Now let's get the length of activity for each user (still using creation times) in days. 

In [90]:
import numpy
user_activity_length = numpy.zeros(len(data))
for i, user in enumerate(data.keys()):
    ctimes = [ctime for ctime, _ in data[user]]
    user_activity_length[i] = (max(ctimes) - min(ctimes))/(86400*1000)
print len(user_activity_length)

0


The histogram shows the number of users whose activity time was within `x` number of days.

In [84]:
earliest = 1e100
latest = 0
for user in data:
    for ctime, _ in data[user]: #ignore last-modified for now
        if ctime < earliest:
            earliest = ctime
        if ctime > latest:
            latest = ctime

print 'Earliest:', convert_time(earliest) 
print 'Latest:', convert_time(latest)
MAXDUR = 300
MINDUR = 150
# get subset of data from "old" users whose earliest creation date is more than MAXDUR days before the end of the dataset
# (giving them a good chance to stay on for more than MINDUR days),
# AND have activity within MAXDUR days if they are active beyond MINDUR (to be fair to recent users)
old_data = {}
for user in data:
    start_time = min([ctime for ctime, _ in data[user]])
    if int((latest-start_time)/(86400.*1000))>MAXDUR:
        post = filter(lambda length: length>MINDUR, 
                      map(lambda (ctime,mtime): (ctime-start_time)/(86400.*1000), data[user]))
        if len(post)>0 and len(filter(lambda length: length<=MAXDUR, post))==0:
            continue  # ignore users who have no activity in the 120-180 period
        old_data[user[3:]] = summaries[user[3:]]
print 'Filtered to', len(old_data), 'users from', len(data)

old_user_activity_length = numpy.zeros(len(old_data))
for i, user in enumerate(old_data.keys()):
    ctimes = [ctime for ctime, _ in data[user[0:2] + '/' + user]]
    old_user_activity_length[i] = (max(ctimes) - min(ctimes))/(86400*1000)

Earliest: 2013-03-27
Latest: 2016-03-10


KeyError: u'42'

In [48]:
def getEarliest(userID): #helper function to get the earliest and latest project creation date for summmaries
    user = summaries[userID]
    earliest = user.keys()[0]
    for proj in user: 
        creationDate = summaries[userID][proj]['**created']
        if  creationDate < earliest: 
            earliest = creationDate
    return earliest
        

In [61]:
def getProjectsInMINDUR(userID): #for summaries
    projDict = {}
    earliest = getEarliest(userID)

    latest = (MINDUR * 86400 * 1000) + earliest 
    user = summaries[userID]
    for proj in user: 
        if summaries[userID][proj]['**created'] >= earliest and summaries[userID][proj]['**created'] <= latest: 
            projDict[proj] = summaries[userID][proj]
    return projDict
                        
                                                                                 

In [62]:
def getData(userID):
    if TASK == 'LANGUAGE': 
        return {userID: summaries[userID]} 
    if TASK == 'RETENTION':
        return {userID : getProjectsInMINDUR(userID)}

In [81]:
# getData('11542')
# #summaries['11542'][u'p020_020_p0930d'].keys()

In [12]:
def getProjects(userID): 
    r = getData(userID)
    projects = r[r.keys()[0]] 
    #print projects[projects.keys()[0]]['**created']

    name_time = [(project, projects[project]['**created']) for project in projects]
    name_time = sorted(name_time, key=lambda t: t[1])  
    proj_list = [summaries[userID][project[0]] for project in name_time]
    return proj_list
    #print proj_list


In [13]:
def getAverageProjectLen(projects):
    num_projects = len(projects)
    total_project_length = 0 #the sum of all the lengths of the project (from date created to date modified)
    
    prev = projects[0]["**created"]
    for project in projects[1:]: 
        curr = project['**created']
        total_project_length += (curr - prev)/(86400.*1000)   #SR: added this to scale to days
        prev = curr 
    return int(total_project_length / float(num_projects))

In [14]:
def numProjects(projects):
    return len(projects)

In [15]:
def percentOnWeekday(projects):
    num_projs = numProjects(projects)
    weekday_projs = 0
    for project in projects: 
        if datetime.datetime.fromtimestamp(project['**created'] / 1e3).weekday() < 6: 
            weekday_projs += 1.0
    return weekday_projs / num_projs * 100

In [16]:
import datetime
def percentOnDay(projects):
    num_projs = len(projects)
    weekday_projs = 0
    dow = [] 
    for i in range(7):
        dow.append(0)
    for project in projects: 
        date = datetime.datetime.fromtimestamp(project['**created'] / 1e3).weekday() 
        dow[date] += 1.0
    for i in range(7):
        dow[i] = dow[i] / num_projs * 100 
    return dow

In [72]:
 def numProjectsInDecile(projects):
    ''' the first decile is the 0th decile (covering from 0-10% of the time), 
    the last decile is the 9th decile'''
    #still not working right
    first= int(projects[0]['**created'])
    #print first
    last = int(projects[len(projects)-1]['**created'])
    decile_width = ((last - earliest) / (86400 * 1000)) / 10
    if decile_width < 1: decile_width = 1
    #print last
    #print decile_width
    decileList = []
    
    for i in range(10): 
        decileList.append(0)
        
    for project in projects: 
        if project['**created'] >= first and project['**created'] <= last: 
            index  = int(((project['**created']-first)/(86400 * 1000)) / (decile_width / 10.))
            #print index
            if index >= 10: 
                decileList[9] = decileList[9] + 1
            else: 
                decileList[index] = decileList[index] + 1
    return decileList
    

In [18]:
def getDecileBoundaries(projects):
    first= projects[len(projects)-1]['**created']
    decile_width = MINDUR * 86400000/ 10 
    decileNums = []
    for i in range(1,11): 
        decileNums.append(first + decile_width*i)
    return decileNums

In [30]:
# users = old_data.keys()[0:10]
# for userID in users:
#     r = getProjects(userID)
#     numProjectsInDecile(r)


In [25]:
#Summary features 

In [41]:
# print summaries['44884']['p021_020_clock']['Screen1']['Blocks']["Orphan Blocks"]#.keys()#['*Top Level Blocks'].keys()#['Active Blocks'].keys()#['Local Variable Names'].keys() #['Local Variable Names']

NO ORPHAN BLOCKS


In [19]:
def getNumScreens(projects):
    numScreens = []
    for project in projects: 
         numScreens.append(project['*Number of Screens'])
    return numScreens
    

In [20]:
def averageNumScreens(projects):
    screenList = getNumScreens(projects)
    return sum(screenList)/ len(screenList)

In [21]:
def getAveNumLocalVariables(projects):
    numScreens = getNumScreens(projects)
    lv_count = 0 
    i = 0 
    while i < len(projects): 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    if projects[i][screenNum]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                        lv_count += len(projects[i][screenNum]['Blocks']['Active Blocks']['Local Variable Names'].keys())
        i+=1
    return lv_count / float(len(projects))

In [22]:
def getAveNumGlobalVariables(projects):
    numScreens = getNumScreens(projects)
    gv_count = 0 
    i = 0 
    while i < len(projects): 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    if projects[i][screenNum]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                        gv_count += len(projects[i][screenNum]['Blocks']['Active Blocks']['Global Variable Names'].keys())
        i+=1
    return gv_count / float(len(projects))

In [23]:
def getAllVariables(projects): 
    '''returns a list with 0th element being the average number of local variables, the 1st element
    being the average number of global variables, and the 2nd element being the average number
    of variables in a project'''
    numScreens = getNumScreens(projects)
    gv_count = 0 
    lv_count = 0 
    i = 0 
    variables = []
    while i < len(projects): 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    if projects[i][screenNum]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                        lv_count += len(projects[i][screenNum]['Blocks']['Active Blocks']['Local Variable Names'].keys())
                        gv_count += len(projects[i][screenNum]['Blocks']['Active Blocks']['Global Variable Names'].keys())
        i+=1
    variables.append(lv_count)
    variables.append(gv_count)
    variables.append(lv_count + gv_count)
    
    return [v/float(len(projects)) for v in variables]

In [24]:
def averageNumBlocks(projects):
    numScreens = getNumScreens(projects)
    totalBlocks = 0 
    i = 0 
    while i < len(projects): 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    if  not isinstance(projects[i][screenNum]['Blocks']['Active Blocks'],unicode):
                        totalBlocks += projects[i][screenNum]['Blocks']['Active Blocks']['*Number of Blocks']
        i+=1
    return totalBlocks / float(len(projects))

In [25]:
def getAverageTLBlocks(projects):
    numScreens = getNumScreens(projects)
    tl_count = 0 
    i = 0 
    while i < len(projects): 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                        tl_count += len(projects[i][screenNum]['Blocks']['*Top Level Blocks'].keys())
        i+=1
    return tl_count / float(len(projects))

In [26]:
# users = lang_data.keys()[0:10000]
# print 'loaded keys'
# i = 0 
# for userID in users:
#     if i % 1000== 0: 
#         print i 
#     r = getProjects(userID)
#     getAverageOrphanBlocks(r)
#     i+=1

In [27]:
def getAverageOrphanBlocks(projects):
    numScreens = getNumScreens(projects)
    tl_count = 0 
    i = 0 
    while i < len(projects): 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    if projects[i][screenNum]['Blocks']['Orphan Blocks'] != "NO ORPHAN BLOCKS": 
                        tl_count += projects[i][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
        i+=1
    return tl_count / float(len(projects))

In [28]:
def getDecileBoundaries(projects):
    earliest = projects[0]["**created"]
    #print (earliest + MINDUR * 86400000)

    decile_width = (latest - earliest)/ 10 
    #print decile_width
    decileNums = []
    for i in range(1,10): 
        decileNums.append(earliest + decile_width*i)
    decileNums.append(projects[len(projects)-1]['**created'])
    return decileNums

In [29]:
def decileNumScreens(projects): 
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    decile_boundaries = getDecileBoundaries(projects)
    
    for project in projects: 
        date = project['**created']
        placed = False
        for boundary in decile_boundaries: 
            if date < boundary and not placed: 
                index = decile_boundaries.index(boundary)
                numScreens = project['*Number of Screens']
                all_deciles[index].insert(0,numScreens)
                placed = True
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles

In [30]:
def decileNumTopLevelBlocks(projects): 
    '''measures how many TLblocks per project in each decile '''
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    decile_boundaries = getDecileBoundaries(projects)
    
    numScreens = getNumScreens(projects)
    for i in range(len(projects)): 
        numTL =0 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    numTL = len(projects[i][screenNum]['Blocks']['*Top Level Blocks'].keys())
        date = projects[i]['**created']
        placed = False
        for boundary in decile_boundaries: 
            if date < boundary and not placed: 
                index = decile_boundaries.index(boundary)
                all_deciles[index].insert(0,numTL)
                placed = True
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles
    

In [31]:
def decileOrphanBlocks(projects): 
    '''measures how many orphan blocks per project in each decile '''
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    decile_boundaries = getDecileBoundaries(projects)
    
    numScreens = getNumScreens(projects)
    for i in range(len(projects)): 
        numOrphan =0 
        for j in range(numScreens[i]):
            screenNum = "Screen"+ str(j + 1)
            if screenNum in projects[i].keys():
                if  not isinstance(projects[i][screenNum]['Blocks'], unicode):
                    if projects[i][screenNum]['Blocks']['Orphan Blocks'] != "NO ORPHAN BLOCKS":
                        numOrphan += len(projects[i][screenNum]['Blocks']['Orphan Blocks'].keys())
        date = projects[i]['**created']
        placed = False
        for boundary in decile_boundaries: 
            if date < boundary and not placed: 
                index = decile_boundaries.index(boundary)
                all_deciles[index].insert(0,numOrphan)
                placed = True
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles
    

In [32]:
# users = lang_data.keys()[0:10000]
# print 'loaded keys'
# i = 0 
# for userID in users:
#     if i % 1000== 0: 
#         print i 
#     r = getProjects(userID)
#     decileOrphanBlocks(r)
#     i+=1

In [57]:
# def decileNumScreens2(projects): 
#     all_deciles = []
#     earliest = projects[0]['**created']
#     latest = projects[len(projects)-1]['**created']
#     decile_width = ((latest - earliest)/ (86400 * 1000.)) / 10

#     for i in range(10):
#          all_deciles.append([])
            
#     decile_boundaries = getDecileBoundaries(projects)
#     for project in projects: 
#         date = project['**created']
#         index  = (date-earliest)/(decile_width / 10)
#         print index
#         #all_deciles[index].insert(0, project['*Number of Screens'])
#     for i in range(10):
#         if len(all_deciles[i]) == 0: 
#             all_deciles[i] = 0
#         else: 
#             all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
#     return all_deciles

In [ ]:
# def decileNumTopLevelBlocks2(userID): 
#     '''measures how many TLblocks per project in each decile '''
#     all_deciles = []
#     for i in range(10):
#          all_deciles.append([])
#     projs = getProjectsNamesInMINDUR(userID)
#     decile_boundaries = getDecileBoundaries(userID)
    
#     for proj in projs: 
#         totalTL =0 
#         numScreens = getNumScreens(userID, proj)
#         for i in range(numScreens):
#             screenNum = "Screen"+ str(i + 1)
#             if screenNum in summaries[userID][proj].keys() and '*Top Level Blocks' in summaries[userID][proj][screenNum]['Blocks']: 
#                 typesOfTL = summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'].keys()
#                 for block in typesOfTL: 
#                     totalTL += summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'][block]
#         date = summaries[userID][proj]['**created']
#         index  = ((date-earliest)/(86400 * 10000))/(MINDUR/10)
#         #print index
#         all_deciles[index].append(totalTL)
#     for i in range(10):
#         if len(all_deciles[i]) == 0: 
#             all_deciles[i] = 0
#         else: 
#             all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
#     return all_deciles
    

In [73]:
# users = lang_data.keys()[0:1]
# print 'loaded keys'
# i = 0 
# for userID in users:
#     if i % 1000== 0: 
#         print i 
#     r = getProjects(userID)
#     print decileNumScreens(r)
#     print decileNumScreens2(r)
#     i+=1

In [ ]:
# def decileOrphanBlocks2(userID): 
#     '''measures how many orphan blocks per project in each decile '''
#     all_deciles = []
#     for i in range(10):
#          all_deciles.append([])
#     projs = getProjectsNamesInMINDUR(userID)
#     decile_boundaries = getDecileBoundaries(userID)
    
#     for proj in projs: 
#         oBlocks =0 
#         numScreens = getNumScreens(userID, proj)
#         for i in range(numScreens):
#             screenNum = "Screen"+ str(i + 1)
#             if screenNum in summaries[userID][proj].keys() and 'Blocks' in summaries[userID][proj][screenNum].keys() and 'Orphan Blocks' in summaries[userID][proj][screenNum]['Blocks']: #occassional error otherwise
#                 if summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks'] != "NO ORPHAN BLOCKS":
#                     try: 
#                         oBlocks += summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
#                     except TypeError: 
#                         print summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
#         date = summaries[userID][proj]['**created']
#         index  = ((date-earliest)/(86400 * 10000))/(MINDUR/10)
#         #print index
#         all_deciles[index].insert(0,oBlocks)
#     for i in range(10):
#         if len(all_deciles[i]) == 0: 
#             all_deciles[i] = 0
#         else: 
#             all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
#     return all_deciles

In [ ]:
# keys = summaries.keys()[0:100]
# for userID in keys: 
#     print decileOrphanBlocks(userID)

In [33]:
def percentProjectsFeatures(projects):
    """return a dictionary mapping each time slot (10%, 20%, etc) to the percentage of projects created
    at that time,
    for a particular user represented as list of (creation times, modified times),
    using only their first MINDUR days of activity"""   
    userDict = {} 
    #note: feature names are for our reference only
    userDict["average length"] = getAverageProjectLen(projects)
    userDict["num projects"] = numProjects(projects)
    
    decileProjects = numProjectsInDecile(projects)
    
    userDict["1st"] = decileProjects[0]
    userDict["2nd"] = decileProjects[1]
    userDict["3rd"] = decileProjects[2]
    userDict["4th"] = decileProjects[3]
    userDict["5th"] = decileProjects[4]
    userDict["6th"] = decileProjects[5]
    userDict["7th"] = decileProjects[6]
    userDict["8th"] = decileProjects[7]
    userDict["9th"] = decileProjects[8]
    userDict["10th"] = decileProjects[9]

    return userDict


In [34]:
def dayAnalysisFeatures(projects): 
    userDict = {} 
    userDict["weekday"] = percentOnWeekday(projects) #
    day = percentOnDay(projects)
    userDict["Monday"] = day[0]
    userDict["Tuesday"] = day[1]
    userDict["Wednesday"] =day[2]
    userDict["Thursday"] = day[3]
    userDict["Friday"] = day[4]
    userDict["Saturday"] = day[5]
    userDict["Sunday"] = day[6]

    return userDict

In [35]:
def summaryOBlockDecile(projects): 
    userDict = {}
    decileOrphanBlock = decileOrphanBlocks(projects)
    userDict["O 1"] = decileOrphanBlock[0]
    userDict["O 2"] = decileOrphanBlock[1]
    userDict["O 3"] = decileOrphanBlock[2]
    userDict["O 4"] = decileOrphanBlock[3]
    userDict["O 5"] = decileOrphanBlock[4]
    userDict["O 6"] = decileOrphanBlock[5]
    userDict["O 7"] = decileOrphanBlock[6]
    userDict["O 8"] = decileOrphanBlock[7]
    userDict["O 9"] = decileOrphanBlock[8]
    userDict["O 10"] = decileOrphanBlock[9]
    return userDict

In [36]:
def summaryDecileTLBlocks(projects): 
    userDict = {}
    decileNumTopLevelBlock = decileNumTopLevelBlocks(projects)
    userDict["TL 1"] = decileNumTopLevelBlock[0]
    userDict["TL 2"] = decileNumTopLevelBlock[1]
    userDict["TL 3"] = decileNumTopLevelBlock[2]
    userDict["TL 4"] = decileNumTopLevelBlock[3]
    userDict["TL 5"] = decileNumTopLevelBlock[4]
    userDict["TL 6"] = decileNumTopLevelBlock[5]
    userDict["TL 7"] = decileNumTopLevelBlock[6]
    userDict["TL 8"] = decileNumTopLevelBlock[7]
    userDict["TL 9"] = decileNumTopLevelBlock[8]
    userDict["TL 10"] = decileNumTopLevelBlock[9]
    return userDict

In [37]:
def summaryDecileNumScreens(projects): 
    userDict = {}
    decileNumS = decileNumScreens(projects)
    userDict["NS 1"] = decileNumS[0]
    userDict["NS 2"] = decileNumS[1]
    userDict["NS 3"] = decileNumS[2]
    userDict["NS 4"] = decileNumS[3]
    userDict["NS 5"] = decileNumS[4]
    userDict["NS 6"] = decileNumS[5]
    userDict["NS 7"] = decileNumS[6]
    userDict["NS 8"] = decileNumS[7]
    userDict["NS 9"] = decileNumS[8]
    userDict["NS 10"] = decileNumS[9]
    return userDict

In [38]:
def summaryAverages(projects):
    userDict = {} 
    userDict["NS"] = averageNumScreens(projects)
    userDict["NB"] = averageNumBlocks(projects)
    userDict["OB"] = getAverageOrphanBlocks(projects)
    
    varList = getAllVariables(projects)
    
    userDict["local vars"] = varList[0]
    userDict["global vars"] = varList[1]
    
    return userDict

Write other feature functions -- perhaps different interval sizes (instead of 10%), proportion of projects rather than number, etc. With some effort, we can get a measure of activity on weekends/evenings as features too. Experiment...

Give each feature function a distinct name, and re-run the above two cells with the function in place of `featurize` to see if the contingency tables change

Remember from the CS111 assignment that we can combine a list of feature functions to give a new feature function, so you should experiment with combinations as well.

In [39]:
def combine_featfuncs(funclist):
    def combined(user):
        basedict = funclist[0](user)
        for f in funclist[1:]:
            basedict.update(f(user))
        return basedict
    return combined

RETENTION:

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import DictVectorizer

user_dicts = []
# i = 0

if TASK == "RETENTION":
    i = 0
    for user in OD:
        i +=1 
        if i % 1000 == 0: 
            print i
        projects = getProjects(user)
        all_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages])
        time_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures])
        code_combined = combine_featfuncs([summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) 
    
        all_features.append(all_combined(projects))
        time_features.append(time_combined(projects))
        code_features.append(code_combined(projects))


    print ('done loading featurizers')
    

    vec = DictVectorizer()
    X1 = vec.fit_transform(all_features) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html
    X2 = vec.fit_transform(time_features)
    X3 = vec.fit_transform(code_features)
    #X = vec.fit_transform(user_dicts) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html

    #X = normalize(X, axis=0, norm='l1') #normalization is bringing the values down significantly 



**Attempt 1: Clustering.** Is there any correlation between the clusters produced by these features, and whether or not a user is retained?

In [82]:
from sklearn.cluster import KMeans
if TASK == 'RETENTION':

    KMeans_model = KMeans(n_clusters=2)
    users_clusterids = KMeans_model.fit_predict(X1)

    from scipy.stats import chi2_contingency
    conting = numpy.zeros((2, 2))  # row = cluster ID (0, 1), col = #days (0 for <=120, 1 for >120)

    for i in range(len(users_clusterids)):
        cluster = users_clusterids[i]
        if old_user_activity_length[i]<=MINDUR:
            conting[cluster, 0] += 1
        else:
            conting[cluster, 1] += 1

# visualize
    print conting

# compute chi2 statistic to get the probability that there is no association between
# the clustering and whether or not the user is active for >120 days
# lower probabilities are better
    _, p, _, _ = chi2_contingency(conting)
    print 'There is a', p, 'probability that the clusters and retention are independent.'

IndexError: index 25324 is out of bounds for axis 0 with size 25324

**Attempt 2: Classification.**

In [78]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn import linear_model

if TASK=='RETENTION':
    y = numpy.array(map(lambda d: int(d>MINDUR), old_user_activity_length))  # labels (0 for less than MINDUR, 1 otherwise)

    percentpos = sum(y)/float(len(y))
    chance = max(percentpos, 1-percentpos)
    print 'Chance is', chance

    print 'Building a model with', X1.shape[1], 'features for', TASK
    all_coefs = [] 
    for trainidx, testidx in StratifiedKFold(y):
        print 'Fold'
        k = 70 #num neighbors
        ytrain = y[trainidx]
        ytest = y[testidx]
        
        X1train = X1[trainidx, :]  # using numpy's smart indexing
        X1test = X1[testidx, :]
        
        X2train = X2[trainidx, :]  # using numpy's smart indexing
        X2test = X2[testidx, :]
    
        X3train = X3[trainidx, :]  # using numpy's smart indexing
        X3test = X3[testidx, :]
    
        KNeighbors_model1 = KNeighborsClassifier(n_neighbors=k)
        logistic_model1 = linear_model.LogisticRegression(fit_intercept= False)

        KNeighbors_model2 = KNeighborsClassifier(n_neighbors=k)
        logistic_model2 = linear_model.LogisticRegression(fit_intercept= False)

        KNeighbors_model3 = KNeighborsClassifier(n_neighbors=k)  
        logistic_model3 = linear_model.LogisticRegression(fit_intercept= False)

        KNeighbors_model1.fit(X1train, ytrain)
        logistic_model1.fit(X1train, ytrain)

        KNN_XtestP1 = KNeighbors_model1.predict(X1test)
        log_XtestP1 = logistic_model1.predict(X1test)
    
        KNeighbors_model2.fit(X2train, ytrain)
        logistic_model2.fit(X2train, ytrain)

        KNN_XtestP2 = KNeighbors_model2.predict(X2test)
        log_XtestP2 = logistic_model2.predict(X2test)
        
        KNeighbors_model3.fit(X3train, ytrain)
        logistic_model3.fit(X3train, ytrain)

        KNN_XtestP3 = KNeighbors_model3.predict(X3test)
        log_XtestP3 = logistic_model3.predict(X3test)
    
    
    
        print k, 'neighbors' 
        print "All features KNN Score: ", KNeighbors_model1.score(X1test, ytest)
        print 'All features KNN f1 score', f1_score(KNN_XtestP1, ytest),'\n'

        print'All features Logistic Regression Score: ', logistic_model1.score(X1test, ytest)
        print 'All features Logistic f1 score', f1_score(log_XtestP1, ytest) 

        print "Time features KNN Score: ", KNeighbors_model2.score(X2test, ytest)
        print 'Time features KNN f1 score', f1_score(KNN_XtestP2, ytest),'\n'

        print'Time features Logistic Regression Score: ', logistic_model2.score(X2test, ytest)
        print 'Time features Logistic f1 score', f1_score(log_XtestP2, ytest) 
        
        print "Code features KNN Score: ", KNeighbors_model2.score(X2test, ytest)
        print 'Code features KNN f1 score', f1_score(KNN_XtestP2, ytest),'\n'

        print'Code features Logistic Regression Score: ', logistic_model3.score(X3test, ytest)
        print 'Code features Logistic f1 score', f1_score(log_XtestP3, ytest) 


Chance is 0.52772073922
Building a model with 55 features for RETENTION
Fold
70 neighbors
All features KNN Score:  0.515280739161
All features KNN f1 score 0.36247467664 

All features Logistic Regression Score:  0.513740819711
All features Logistic f1 score 0.217051306504
Time features KNN Score:  0.507936507937
Time features KNN f1 score 0.342305256491 

Time features Logistic Regression Score:  0.526533996683
Time features Logistic f1 score 0.12327264751
Code features KNN Score:  0.507936507937
Code features KNN f1 score 0.342305256491 

Code features Logistic Regression Score:  0.520374318882
Code features Logistic f1 score 0.165670719143
Fold


KeyboardInterrupt: 

LANGUAGE:
    

In [41]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import DictVectorizer
all_features = []
time_features = [] 
code_features = []

users = lang_data.keys()
print ('done loading keys')
i = 0 
for user in users:
    i +=1 
    if i % 1000 == 0: 
        print i
    projects = getProjects(user)
    all_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages])
    time_combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures])
    code_combined = combine_featfuncs([summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) 

    all_features.append(all_combined(projects))
    time_features.append(time_combined(projects))
    code_features.append(code_combined(projects))
    
# for userID in lang_data: #user is just the user's id num 
#     combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, decileFeatures ,summaryNumScreensDecile,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) #a function object is being returned here?? 
#     user_dicts.append(combined(userID))


print ('done loading featurizers')
    

vec = DictVectorizer()
X1 = vec.fit_transform(all_features) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html
X2 = vec.fit_transform(time_features)
X3 = vec.fit_transform(code_features)
#X = normalize(X, axis=0, norm='l1') #normalization is bringing the values down significantly 



done loading keys
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
done loading featurizers


NameError: name 'X' is not defined

In [ ]:
# print len(data)
# print len(lang_data)
# #what y is
# print [lang_dict[user_langs[userID]] for userID in lang_data]

In [44]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn.metrics import confusion_matrix


print old_user_activity_length[0:100]
if TASK=='LANGUAGE':
    y =  numpy.array([lang_dict[user_langs[userID]] for userID in lang_data]) # labels

percentpos = sum(y)/float(len(y))
chance = max(percentpos, 1-percentpos)
print 'Chance is', chance

#print 'Building a model with', X1.shape[1], 'features for', TASK
all_coefs = [] 
for trainidx, testidx in StratifiedKFold(y):
    print 'Fold'
    k = 70 #num neighbors
    X1train = X1[trainidx, :]  # using numpy's smart indexing
    X1test = X1[testidx, :]
    ytrain = y[trainidx]
    ytest = y[testidx]
    
    X2train = X2[trainidx, :]  # using numpy's smart indexing
    X2test = X2[testidx, :]
    
    X3train = X3[trainidx, :]  # using numpy's smart indexing
    X3test = X3[testidx, :]
    
    KNeighbors_model1 = KNeighborsClassifier(n_neighbors=k)  
    KNeighbors_model2 = KNeighborsClassifier(n_neighbors=k)  
    KNeighbors_model3 = KNeighborsClassifier(n_neighbors=k)  
    
    print('models created')

    KNeighbors_model1.fit(X1train, ytrain)
    print 'model1 fit'
    KNeighbors_model2.fit(X2train, ytrain)
    print 'model2 fit'
    KNeighbors_model3.fit(X3train, ytrain)  
    print ('all models fit')
    
    KNN_XtestP1 = KNeighbors_model1.predict(X1test)
    print 'model1 predicted'
    KNN_XtestP2 = KNeighbors_model2.predict(X2test)
    print 'model2 predicted'
    KNN_XtestP3 = KNeighbors_model3.predict(X3test)
    print 'models predicted \n'
    
    print k, 'neighbors' 
    print "All features KNN Score: ", KNeighbors_model1.score(X1test, ytest)
    print 'All features KNN f1 score', f1_score(KNN_XtestP1, ytest),'\n'

    print "time features KNN Score: ", KNeighbors_model2.score(X2test, ytest)
    print 'time features KNN f1 score', f1_score(KNN_XtestP2, ytest),'\n'

    print "code features KNN Score: ", KNeighbors_model3.score(X3test, ytest)
    print 'code features KNN f1 score', f1_score(KNN_XtestP3, ytest),'\n'
    
    cnf_matrix = confusion_matrix(ytest, KNN_XtestP1)
    print cnf_matrix


[ 209.  237.  238.  416.  238.  496.  131.  115.  386.   77.  474.   73.
  393.  357.  352.  259.  288.   80.  671.  523.   21.  448.  399.  110.
  111.   87.   82.  205.  176.  296.  808.  539.  403.   87.  274.  118.
  275.  734.  706.  201.  276.  315.  238.  361.  394.  320.   77.  116.
   40.   37.  525.   84.  739.   25.   59.  119.   91.  766.  103.  232.
  107.   44.  298.   13.   83.  141.  436.  270.  471.  111.  739.   87.
   55.  477.  270.  209.   63.   13.   49.  104.  424.  516.  216.  228.
  269.   98.  119.   84.  111.  283.  736.  268.  114.   86.  678.  111.
  122.  130.  228.   97.]
Chance is 44.59384683
Fold
models created
models fit
models predicted 

70 neighbors
All features KNN Score:  0.592261121348
All features KNN f1 score

/Users/emmalurie/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/Users/emmalurie/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


 0.681591036075 

time features KNN Score:  0.589892076862
time features KNN f1 score 0.680842365005 

code features KNN Score: 

/Users/emmalurie/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


 0.504211634641
code features KNN f1 score 0.593063706722 

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


...

/Users/emmalurie/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


 0.591509309389
time features KNN f1 score 0.680737852999 

code features KNN Score: 

/Users/emmalurie/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


 0.508253004093
code features KNN f1 score 0.596349186174 

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


/Users/emmalurie/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
